# Preparing single-cell data for a benchmark (out-of-core with Zarr)

This is an alternative to the `00_prepare_dataset.ipynb` notebook that uses [Zarr](https://zarr.dev) for an out-of-core pre-processing workflow.
This is useful if you don't have enough RAM to hold the single-cell expression data in memory (not enough RAM).

At the moment this assumes that normalised expression data (not raw counts) is extracted.

**Please use a separate conda environment for this:**

```
conda create -n zarr python=3.9
conda activate zarr
pip install zarr scanpy numpy==1.26.4
```

In [1]:
import scanpy as sc, anndata as ad, numpy as np, os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
from tqdm import tqdm

dataset_name = 'Suo'
output_path = './data'
batch_size = 1000
colname = 'cell_type'

if not os.path.exists(output_path):
    os.mkdir(output_path)

/Users/davidnovak/miniforge3/envs/ViVAE/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/Users/davidnovak/miniforge3/envs/ViVAE/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/Users/davidnovak/miniforge3/envs/ViVAE/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/Users/davidnovak/miniforge3/envs/ViVAE/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/Users/davidnovak/miniforge3/envs/ViVAE/lib/python3.11/site-packages/anndata/utils.py:

In [13]:
%%bash
wget -O ./scrnaseq.h5ad https://datasets.cellxgene.cziscience.com/62b18ca2-6956-49f1-8b6d-0885fb38e1ac.h5ad >/dev/null 2>&1

In [ ]:
hd = sc.read_h5ad('./scrnaseq.h5ad')
hd.write_zarr('scrnaseq.zarr')
del hd

In [10]:
adata = ad.read_zarr('scrnaseq.zarr')

In [12]:
n_obs, n_vars = adata.shape
print(f'{n_obs} cells, {n_vars} genes')

908046 cells, 33145 genes


In [15]:
mean = np.zeros(n_vars)
M2 = np.zeros(n_vars)
count = 0

for start in tqdm(range(0, n_obs, batch_size)):
    end = min(start + batch_size, n_obs)
    X = adata.X[start:end]
    if not isinstance(X, np.ndarray):
        X = X.toarray()
    count += X.shape[0]
    delta = X - mean
    mean += np.sum(delta, axis=0) / count
    delta2 = X - mean
    M2 += np.sum(delta * delta2, axis=0)

100%|██████████| 909/909 [03:22<00:00,  4.48it/s]


In [16]:
std = np.sqrt(M2 / (count - 1))
std[std == 0] = 1.0  # prevent division by zero

In [18]:
ipca = IncrementalPCA(n_components=100)

In [20]:
print('Fitting Incremental PCA...')
for start in tqdm(range(0, n_obs, batch_size)):
    end = min(start + batch_size, n_obs)
    X = adata.X[start:end]
    if not isinstance(X, np.ndarray):
        X = X.toarray()
    X_scaled = (X - mean) / std
    ipca.partial_fit(X_scaled)

Fitting Incremental PCA...


100%|██████████| 909/909 [5:04:09<00:00, 20.08s/it]    


In [21]:
pcs = []
print('Transforming data into PCA space...')
for start in tqdm(range(0, n_obs, batch_size)):
    end = min(start + batch_size, n_obs)
    X = adata.X[start:end]
    if not isinstance(X, np.ndarray):
        X = X.toarray()
    X_scaled = (X - mean) / std
    pcs.append(ipca.transform(X_scaled))

pcs = np.concatenate(pcs, axis=0)

Transforming data into PCA space...


100%|██████████| 909/909 [05:18<00:00,  2.86it/s]


In [22]:
np.save(os.path.join(output_path, f'{dataset_name}_input.npy'), pcs, allow_pickle=True)
print(f'Saved {pcs.shape[0]}-by-{pcs.shape[1]} PC matrix')

Saved 908046-by-100 PC matrix


In [3]:
adata = ad.read_zarr('scrnaseq.zarr')

In [5]:
unassigned = []

labels = adata.obs[colname]
np.save(os.path.join(output_path, f'{dataset_name}_labels.npy'), labels, allow_pickle=True)
np.save(os.path.join(output_path, f'{dataset_name}_unassigned.npy'), unassigned, allow_pickle=True)
print(f'Saved {len(labels)}-label vector with {len(np.unique(labels))} unique labels')

Saved 908046-label vector with 66 unique labels


**At this point, restart the kernel, switch to your ViScore conda environment, run the first code cell and return here.**

In [3]:
import viscore as vs, numpy as np, os

pcs = np.load(os.path.join(output_path, f'{dataset_name}_input.npy'), allow_pickle=True)
k = 150

knn = vs.make_knn(x=pcs, k=k, fname=os.path.join(output_path, f'{dataset_name}_knn.npy'), verbose=False)
print(f'Saved {k}-nearest-neighbour graph')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Saved 150-nearest-neighbour graph


In [6]:
pcs_d = vs.smooth(pcs, knn, k=100, coef=1., n_iter=1)
np.save(os.path.join(output_path, f'{dataset_name}_inpu_denoised.npy'), pcs_d, allow_pickle=True)
print('Saved denoised PC matrix')

Saved denoised PC matrix


In [ ]:
knn = vs.make_knn(x=pcs_d, k=150, fname=os.path.join(output_path, f'{dataset_name}_knn_denoised.npy'), verbose=False)
print(f'Saved denoised {k}-nearest-neighbour graph')